### Connecting to DB and Viewing Table Details

In [36]:
import sqlalchemy as db
import pandas as pd

In [5]:
engine = db.create_engine('sqlite:///data/census.sqlite')
connection = engine.connect()
metadata = db.MetaData()
census = db.Table('census', metadata, autoload = True, autoload_with = engine)

In [6]:
print(census.columns.keys())

['state', 'sex', 'age', 'pop2000', 'pop2008']


In [16]:
print(repr(metadata.tables['census']))

Table('census', MetaData(bind=None), Column('state', VARCHAR(length=30), table=<census>), Column('sex', VARCHAR(length=1), table=<census>), Column('age', INTEGER(), table=<census>), Column('pop2000', INTEGER(), table=<census>), Column('pop2008', INTEGER(), table=<census>), schema=None)


### Querying

In [23]:
query = db.select([census])

In [29]:
ResultProxy = connection.execute(query)

In [25]:
ResultSet = ResultProxy.fetchall()

In [26]:
ResultSet[:3]

[('Illinois', 'M', 0, 89600, 95012),
 ('Illinois', 'M', 1, 88445, 91829),
 ('Illinois', 'M', 2, 88729, 89547)]

#### Dealing with Large ResultSet

In [34]:
ResultProxy = connection.execute(query)
partial_results = ResultProxy.fetchmany(50)
print(len(partial_results))
ResultProxy.close()

50


#### Converting to dataframe

In [39]:
df = pd.DataFrame(ResultSet)
df.columns = ResultSet[0].keys()
df.head()

,state,sex,age,pop2000,pop2008
0,Illinois,M,0,89600,95012
1,Illinois,M,1,88445,91829
2,Illinois,M,2,88729,89547
3,Illinois,M,3,88868,90037
4,Illinois,M,4,91947,91111


### Filtering Data

#### where

In [47]:
query = db.select([census]).where(census.columns.sex=='F')
ResultProxy = connection.execute(query)
ResultSet = ResultProxy.fetchall()
ResultProxy.close()
ResultSet[:5]

[('Illinois', 'F', 0, 85910, 90286),
 ('Illinois', 'F', 1, 84396, 88126),
 ('Illinois', 'F', 2, 84764, 86291),
 ('Illinois', 'F', 3, 85598, 85170),
 ('Illinois', 'F', 4, 87729, 86959)]

#### in

In [55]:
query = db.select([census]).where(census.columns.state.in_(['Texas','New York']))
ResultProxy = connection.execute(query)
ResultSet = ResultProxy.fetchall()
ResultProxy.close()
ResultSet[:5]

[('New York', 'M', 0, 126237, 128088),
 ('New York', 'M', 1, 124008, 125649),
 ('New York', 'M', 2, 124725, 121615),
 ('New York', 'M', 3, 126697, 120580),
 ('New York', 'M', 4, 131357, 122482)]

#### and, or, not

In [49]:
query = db.select([census]).where(db.and_(census.columns.state=='California', census.columns.sex!='M'))
ResultProxy = connection.execute(query)
ResultSet = ResultProxy.fetchall()
ResultProxy.close()
ResultSet[:5]

[('California', 'F', 0, 239605, 274356),
 ('California', 'F', 1, 236543, 269140),
 ('California', 'F', 2, 240010, 262556),
 ('California', 'F', 3, 245739, 259061),
 ('California', 'F', 4, 254522, 255544)]

#### order by

In [51]:
query = db.select([census]).order_by(db.desc(census.columns.state))
ResultProxy = connection.execute(query)
ResultSet = ResultProxy.fetchall()
ResultProxy.close()
ResultSet[:5]

[('Wyoming', 'M', 0, 3236, 4066),
 ('Wyoming', 'M', 1, 3245, 4159),
 ('Wyoming', 'M', 2, 3102, 4058),
 ('Wyoming', 'M', 3, 3103, 3745),
 ('Wyoming', 'M', 4, 3166, 3633)]

#### functions

In [52]:
query = db.select([db.func.sum(census.columns.pop2008)])
ResultProxy = connection.execute(query)
ResultSet = ResultProxy.fetchall()
ResultProxy.close()
ResultSet

[(302876613,)]

#### group by

In [54]:
query = db.select([db.func.sum(census.columns.pop2008).label('pop2008'),census.columns.sex]).group_by(census.columns.sex)
ResultProxy = connection.execute(query)
ResultSet = ResultProxy.fetchall()
ResultProxy.close()
df = pd.DataFrame(ResultSet)
df.columns = ResultSet[0].keys()
df

,pop2008,sex
0,153959198,F
1,148917415,M


#### distinct

In [57]:
query = db.select([census.columns.state.distinct()])
ResultProxy = connection.execute(query)
ResultSet = ResultProxy.fetchall()
ResultProxy.close()
ResultSet[:5]

[('Illinois',),
 ('New Jersey',),
 ('District of Columbia',),
 ('North Dakota',),
 ('Florida',)]

#### case & cast

In [60]:
female_pop = db.func.sum(db.case([(census.columns.sex == 'F', census.columns.pop2000)],else_=0))
total_pop = db.cast(db.func.sum(census.columns.pop2000), db.Float)
query = db.select([female_pop/total_pop * 100])
result = connection.execute(query).scalar()
print(result)

51.09467432293413


#### joins

In [62]:
state_fact = db.Table('state_fact', metadata, autoload=True,autoload_with=engine)

##### Automatic Join

In [66]:
query = db.select([census, state_fact])
results = connection.execute(query).fetchall()
df = pd.DataFrame(results)
df.columns = results[0].keys()
df.head()

,state,sex,age,pop2000,pop2008,id,name,abbreviation,country,type,...,occupied,notes,fips_state,assoc_press,standard_federal_region,census_region,census_region_name,census_division,census_division_name,circuit_court
0,Illinois,M,0,89600,95012,13,Illinois,IL,USA,state,...,occupied,,17,Ill.,V,2,Midwest,3,East North Central,7
1,Illinois,M,0,89600,95012,30,New Jersey,NJ,USA,state,...,occupied,,34,N.J.,II,1,Northeast,2,Mid-Atlantic,3
2,Illinois,M,0,89600,95012,34,North Dakota,ND,USA,state,...,occupied,,38,N.D.,VIII,2,Midwest,4,West North Central,8
3,Illinois,M,0,89600,95012,37,Oregon,OR,USA,state,...,occupied,,41,Ore.,X,4,West,9,Pacific,9
4,Illinois,M,0,89600,95012,51,Washington DC,DC,USA,capitol,...,occupied,,11,,III,3,South,5,South Atlantic,D.C.


##### Manual Join

In [68]:
query = db.select([census, state_fact])
query = query.select_from(census.join(state_fact, census.columns.state == state_fact.columns.name))
results = connection.execute(query).fetchall()
df = pd.DataFrame(results)
df.columns = results[0].keys()
df.head()

,state,sex,age,pop2000,pop2008,id,name,abbreviation,country,type,...,occupied,notes,fips_state,assoc_press,standard_federal_region,census_region,census_region_name,census_division,census_division_name,circuit_court
0,Illinois,M,0,89600,95012,13,Illinois,IL,USA,state,...,occupied,,17,Ill.,V,2,Midwest,3,East North Central,7
1,Illinois,M,1,88445,91829,13,Illinois,IL,USA,state,...,occupied,,17,Ill.,V,2,Midwest,3,East North Central,7
2,Illinois,M,2,88729,89547,13,Illinois,IL,USA,state,...,occupied,,17,Ill.,V,2,Midwest,3,East North Central,7
3,Illinois,M,3,88868,90037,13,Illinois,IL,USA,state,...,occupied,,17,Ill.,V,2,Midwest,3,East North Central,7
4,Illinois,M,4,91947,91111,13,Illinois,IL,USA,state,...,occupied,,17,Ill.,V,2,Midwest,3,East North Central,7


### Creating and Inserting Data into Tables

#### Creating Database and Table

In [102]:
engine = db.create_engine('sqlite:///data/test.sqlite')

In [103]:
connection = engine.connect()
metadata = db.MetaData()

In [104]:
emp = db.Table('emp', metadata,
              db.Column('Id', db.Integer()),
              db.Column('name', db.String(255), nullable = False),
              db.Column('salary', db.Float(), default = 100.0),
              db.Column('active', db.Boolean(), default = True)
              )
metadata.create_all(engine)

#### Inserting Data

In [105]:
# Inserting one record at a time
query = db.insert(emp).values(Id=1,name='naveen', salary=60000.0, active = True)
ResultProxy = connection.execute(query)

In [106]:
# Inserting many records at a time
query = db.insert(emp)
value_list = [
    {'Id': '2', 'name': 'ram', 'salary': 80000, 'active':False},
    {'Id': '3', 'name': 'ramesh', 'salary': 70000, 'active':True},
]
ResultProxy = connection.execute(query, value_list)

In [107]:
results = connection.execute(db.select([emp])).fetchall()
df = pd.DataFrame(results)
df.columns = results[0].keys()
df

,Id,name,salary,active
0,1,naveen,60000.0,True
1,2,ram,80000.0,False
2,3,ramesh,70000.0,True


### Updating data in Databases

In [82]:
query = db.update(emp).values(salary = 100000)
query = query.where(emp.columns.Id == 1)
results = connection.execute(query)

In [83]:
results = connection.execute(db.select([emp])).fetchall()
df = pd.DataFrame(results)
df.columns = results[0].keys()
df

,Id,name,salary,active
0,1,naveen,100000.0,True
1,1,naveen,100000.0,True
2,2,ram,80000.0,False
3,3,ramesh,70000.0,True


### Delete rows in a Table

In [92]:
query = db.delete(emp)
query = query.where(emp.columns.salary<100000)
results = connection.execute(query)

In [93]:
results = connection.execute(db.select([emp])).fetchall()
df = pd.DataFrame(results)
df.columns = results[0].keys()
df

,Id,name,salary,active
0,1,naveen,100000.0,True
1,1,naveen,100000.0,True


### Dropping a Table

In [100]:
engine = db.create_engine('sqlite:///data/test.sqlite')
metadata = db.MetaData()
connection = engine.connect()
emp = db.Table('emp', metadata, autoload=True, autoload_with=engine)

results = connection.execute(db.select([emp])).fetchall()
df = pd.DataFrame(results)
df.columns = results[0].keys()
df.head(4)

NoSuchTableError: emp

In [95]:
emp.drop(engine)

In [101]:
metadata.drop_all(engine)